In [6]:
import os
os.environ["KAGGLE_USERNAME"] = "magdhndi"
os.environ["KAGGLE_KEY"] = "9b6c8953fb75d807a407f863ae22edc6"

# Download the datasets (replace with the actual Kaggle dataset URLs)
!kaggle datasets download -d deadskull7/fer2013
!kaggle datasets download -d jonathanoheix/affectnet
!kaggle datasets download -d ahmedhamada/stress-faces
# RAF-DB needs manual download as per the error message

# Unzip the downloaded datasets
!unzip fer2013.zip -d data/fer2013
!unzip affectnet.zip -d data/affectnet
!unzip stress-faces.zip -d data/stress-faces

# Create the data directory if it doesn't exist
!mkdir -p data/raf-db

# Print confirmation messages
print("Datasets downloaded and extracted successfully (except RAF-DB).")
print("Please manually download and place RAF-DB into the data/raf-db directory.")

# Verify if the datasets have been downloaded correctly (optional)
import os
datasets = ["fer2013", "affectnet", "stress-faces", "raf-db"]
for dataset in datasets:
    path = f"data/{dataset}"
    if os.path.exists(path) and os.listdir(path):
        print(f"✅ {dataset} verification successful")
    elif dataset == "raf-db":
        print(f"⚠️ {dataset} requires manual download and placement.")
    else:
        print(f"❌ {dataset} verification failed. Directory not found or empty.")


Dataset URL: https://www.kaggle.com/datasets/deadskull7/fer2013
License(s): CC0-1.0
fer2013.zip: Skipping, found more recently modified local copy (use --force to force download)
403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/datasets/metadata/jonathanoheix/affectnet
403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/datasets/metadata/ahmedhamada/stress-faces
Archive:  fer2013.zip
  inflating: data/fer2013/fer2013.csv  
unzip:  cannot find or open affectnet.zip, affectnet.zip.zip or affectnet.zip.ZIP.
unzip:  cannot find or open stress-faces.zip, stress-faces.zip.zip or stress-faces.zip.ZIP.
Datasets downloaded and extracted successfully (except RAF-DB).
Please manually download and place RAF-DB into the data/raf-db directory.
✅ fer2013 verification successful
❌ affectnet verification failed. Directory not found or empty.
❌ stress-faces verification failed. Directory not found or empty.
⚠️ raf-db requires manual download and placement.


In [1]:
# @title <<< Stage 1 / Task 1.1: Dataset Verification >>>

import os
from pathlib import Path
import pandas as pd



os.environ["KAGGLE_USERNAME"] = "magdhndi"
os.environ["KAGGLE_KEY"] = "9b6c8953fb75d807a407f863ae22edc6"

# Directories defined previously
DATA_DIR = Path("./data")
FER2013_DIR = DATA_DIR / "fer2013"
AFFECTNET_DIR = DATA_DIR / "affectnet"
RAFDB_DIR = DATA_DIR / "raf-db" # Assumed manual download
STRESS_FACES_DIR = DATA_DIR / "stress-faces"

def verify_dataset(dir_path, expected_files_or_patterns):
    """Checks if a directory exists and contains expected files/patterns."""
    if not dir_path.exists() or not dir_path.is_dir():
        print(f"❌ Verification Failed: Directory not found - {dir_path}")
        return False
    print(f"✔️ Directory Found: {dir_path}")

    found_all = True
    for pattern in expected_files_or_patterns:
        matches = list(dir_path.glob(pattern))
        if not matches:
            print(f"    ❌ Expected file/pattern not found: '{pattern}'")
            found_all = False
        else:
             print(f"    ✔️ Found {len(matches)} match(es) for: '{pattern}'")
    return found_all

print("--- Verifying Datasets ---")
verification_checks = {
    FER2013_DIR: ["fer2013.csv"],
    AFFECTNET_DIR: ["training.csv", "images/*.jpg"], # Check for CSV and some images
    STRESS_FACES_DIR: ["Level_*/*.png"], # Check for level dirs and png images
    RAFDB_DIR: ["basic/Image/original/*.jpg", "basic/EmoLabel/list_patition_label.txt"] # Check structure if manually downloaded
}

all_verified = True
for dir_path, patterns in verification_checks.items():
    print(f"\nVerifying {dir_path.name}...")
    if not verify_dataset(dir_path, patterns):
        all_verified = False
        if dir_path == RAFDB_DIR:
            print("   -> Note: RAF-DB requires manual download and placement.")

if all_verified:
    print("\n✅ All specified datasets seem to be present.")
else:
    print("\n⚠️ Some datasets are missing or incomplete. Please check downloads/paths.")

# Further loading logic depends on which datasets are successfully verified and needed.
# We'll continue using Stress Faces as the primary example for simplicity.

--- Verifying Datasets ---

Verifying fer2013...
✔️ Directory Found: data/fer2013
    ✔️ Found 1 match(es) for: 'fer2013.csv'

Verifying affectnet...
❌ Verification Failed: Directory not found - data/affectnet

Verifying stress-faces...
❌ Verification Failed: Directory not found - data/stress-faces

Verifying raf-db...
✔️ Directory Found: data/raf-db
    ❌ Expected file/pattern not found: 'basic/Image/original/*.jpg'
    ❌ Expected file/pattern not found: 'basic/EmoLabel/list_patition_label.txt'
   -> Note: RAF-DB requires manual download and placement.

⚠️ Some datasets are missing or incomplete. Please check downloads/paths.


In [10]:
#!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.7 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is inco

In [14]:
#!pip install --upgrade --force-reinstall mediapipe

  Using cached mediapipe-0.10.21-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached protobuf-4.25.7-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached sounddevice-0.5.1-py3-none-any.whl.metadata (1.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 7.1 MB/s eta 0:00:00
Using cached mediapipe-0.10.21-cp311-cp311-manylinux_2_28_x86_64.whl (35.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.9 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached protobuf-4.25.7-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
Using cached sounddevice-0.5.1-py3-none-any.whl (32 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# @title <<< Stage 1 / Task 1.2: Face Detection & Landmarks (MediaPipe Setup) >>>

import mediapipe as mp
import numpy as np
import cv2

# Ensure MediaPipe objects are initialized (code from previous step)
mp_face_mesh = mp.solutions.face_mesh

# Explicitly specify the model path if necessary
# For example:
# model_path = "path/to/your/face_landmark.tflite"
# face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, model_path=model_path)

# If you are using the default model, try reinstalling mediapipe:
# !pip install --upgrade --force-reinstall mediapipe

face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True, # Use 478 landmarks
    min_detection_confidence=0.5)

# Re-define or import the detect_face_and_landmarks_mediapipe function here
# (Assuming it's available from the previous execution cell)
# [Insert the detect_face_and_landmarks_mediapipe function code here if running standalone]

print("✔️ MediaPipe FaceMesh (478 landmarks) initialized.")
# Example Usage (Optional):
# img_path_example = list(STRESS_FACES_DIR.glob('Level_0/*.png'))[0]
# img_example = cv2.imread(str(img_path_example))
# landmarks, bbox, face_crop = detect_face_and_landmarks_mediapipe(img_example)
# if landmarks is not None:
#   print(f"Detected face and {landmarks.shape[0]} landmarks.")
# else:
#   print("No face detected in example image.")

/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.6.0, so it will not be used.
  warnings.warn(


✔️ MediaPipe FaceMesh (478 landmarks) initialized.


In [3]:
# @title <<< Stage 1 / Task 1.3: rPPG Estimation (CHROM Implementation) >>>

import numpy as np
import scipy.signal
from scipy.signal import butter, filtfilt
from scipy.fft import fft, fftfreq

# --- Constants for rPPG ---
# Typical physiological heart rate range in Hz
MIN_HR_HZ = 0.75 # 45 bpm
MAX_HR_HZ = 3.0  # 180 bpm
# ROI selection using MediaPipe landmarks (approximate forehead/cheeks)
# These indices might need adjustment based on the specific MediaPipe model version (468 vs 478)
# Example indices for cheeks (adjust as needed):
LEFT_CHEEK_IDXS = [234, 127, 162, 21, 54, 103, 67, 109] # Example set
RIGHT_CHEEK_IDXS = [454, 356, 389, 251, 284, 332, 297, 338] # Example set
FOREHEAD_IDXS = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 447, 266, 229, 228, 29] # Example

def select_rois(landmarks_sequence, frame_shape, roi_indices):
    """Selects pixel coordinates within ROIs based on landmarks."""
    # landmarks_sequence: list/array of landmark arrays [(N_lm, 3), ...]
    # frame_shape: (height, width)
    # roi_indices: list of landmark indices defining the ROI polygon(s)

    # This is simplified: For robustness, average coordinates within polygons
    # defined by roi_indices for each frame. Here, we just take the mean
    # location of the indices for simplicity of demonstration.
    # A better approach involves creating masks.

    h, w = frame_shape
    roi_pixels_sequence = []

    for landmarks in landmarks_sequence:
        if landmarks is None:
            roi_pixels_sequence.append(None) # Handle frames where landmarks failed
            continue

        # Get coordinates for ROI indices, scale to pixel values
        roi_coords = landmarks[roi_indices, :2] * [w, h]
        roi_coords = roi_coords.astype(int)

        # Basic mean value within the bounding box of ROI points (simplistic)
        # TODO: Implement proper masking for better ROI analysis
        x_min, y_min = roi_coords.min(axis=0)
        x_max, y_max = roi_coords.max(axis=0)
        # Ensure bounds are valid
        x_min, y_min = max(0, x_min), max(0, y_min)
        x_max, y_max = min(w - 1, x_max), min(h - 1, y_max)

        if x_max > x_min and y_max > y_min:
             roi_pixels_sequence.append({'x_min': x_min, 'y_min': y_min, 'x_max': x_max, 'y_max': y_max})
        else:
             roi_pixels_sequence.append(None)

    return roi_pixels_sequence


def extract_mean_rgb(frames_sequence, rois_sequence):
    """Extracts mean RGB values from ROIs over a sequence of frames."""
    mean_rgb_signal = []
    if len(frames_sequence) != len(rois_sequence):
         raise ValueError("Frames and ROIs sequence lengths mismatch.")

    for frame, roi_bbox in zip(frames_sequence, rois_sequence):
        if frame is None or roi_bbox is None:
            # Handle missing data, e.g., append NaN or previous value
            mean_rgb_signal.append([np.nan, np.nan, np.nan])
            continue

        x_min, y_min = roi_bbox['x_min'], roi_bbox['y_min']
        x_max, y_max = roi_bbox['x_max'], roi_bbox['y_max']

        roi_patch = frame[y_min:y_max, x_min:x_max]
        if roi_patch.size == 0:
            mean_rgb_signal.append([np.nan, np.nan, np.nan])
            continue

        # Calculate mean R, G, B in the ROI
        mean_rgb = np.mean(roi_patch, axis=(0, 1))
        mean_rgb_signal.append(mean_rgb)

    return np.array(mean_rgb_signal) # Shape (num_frames, 3)


def bandpass_filter(signal, lowcut, highcut, fs, order=5):
    """Applies a Butterworth bandpass filter."""
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    # Handle potential edge cases for low/high frequencies
    low = max(low, 1e-6)  # Prevent division by zero or negative frequencies
    high = min(high, 1.0 - 1e-6) # Ensure high is less than 1.0

    if low >= high:
        print(f"Warning: Low cut ({lowcut} Hz) >= High cut ({highcut} Hz). Skipping filter.")
        return signal # Return unfiltered signal or handle as error

    try:
      b, a = butter(order, [low, high], btype='band')
      filtered_signal = filtfilt(b, a, signal, axis=0)
      return filtered_signal
    except ValueError as e:
       print(f"Error during filtering: {e}. Signal length might be too short for filter order.")
       # Return original signal or NaN array of same shape
       return signal # Or np.full_like(signal, np.nan)


def estimate_rppg_chrom(rgb_signal, fs):
    """
    Estimates heart rate from mean RGB signal using the CHROM method.

    Args:
        rgb_signal (np.ndarray): Array of mean RGB values per frame, shape (num_frames, 3).
        fs (float): Sampling frequency (frames per second).

    Returns:
        float: Estimated heart rate in BPM, or None if estimation fails.
    """
    if rgb_signal is None or len(rgb_signal) < int(fs * 2): # Need at least a few seconds
        print("Warning: Insufficient signal length for rPPG.")
        return None
    # Handle NaNs if any (e.g., simple interpolation or skip)
    if np.isnan(rgb_signal).any():
        print("Warning: NaN values detected in RGB signal, attempting interpolation.")
        # Simple linear interpolation - more robust methods might be needed
        for i in range(3): # Interpolate each channel R, G, B
             channel = rgb_signal[:, i]
             nan_indices = np.isnan(channel)
             if np.all(nan_indices): return None # Cannot interpolate if all are NaN
             indices = lambda z: z.nonzero()[0]
             channel[nan_indices] = np.interp(indices(nan_indices), indices(~nan_indices), channel[~nan_indices])
        if np.isnan(rgb_signal).any(): return None # Failed interpolation

    # 1. Bandpass filter RGB signals
    filtered_rgb = bandpass_filter(rgb_signal, MIN_HR_HZ, MAX_HR_HZ, fs)
    if filtered_rgb is None: return None

    # 2. Calculate CHROM signal (Simplified version)
    # Xs = 3*Rn - 2*Gn (approximation of projection orthogonal to specular)
    # Ys = 1.5*Rn + Gn - 1.5*Bn (approximation)
    # More standard: Use standardized signals
    mean_rgb = np.mean(filtered_rgb, axis=0)
    std_rgb = np.std(filtered_rgb, axis=0)
    if np.any(std_rgb == 0): return None # Avoid division by zero
    normalized_rgb = (filtered_rgb - mean_rgb) / std_rgb

    Rn, Gn, Bn = normalized_rgb[:, 0], normalized_rgb[:, 1], normalized_rgb[:, 2]
    Xs = 3 * Rn - 2 * Gn
    Ys = 1.5 * Rn + Gn - 1.5 * Bn

    # Alpha calculation (ratio of std deviations)
    alpha = np.std(Xs) / (np.std(Ys) + 1e-6) # Add epsilon for stability
    chrom_signal = Xs - alpha * Ys

    # 3. FFT to find dominant frequency
    N = len(chrom_signal)
    if N == 0: return None

    yf = fft(chrom_signal)
    xf = fftfreq(N, 1 / fs) # Frequency bins

    # Consider only positive frequencies within the valid HR range
    freq_mask = (xf >= MIN_HR_HZ) & (xf <= MAX_HR_HZ)
    if not np.any(freq_mask):
         print("Warning: No frequency components found in the valid HR range.")
         return None

    fft_power = np.abs(yf[freq_mask])**2
    valid_freqs = xf[freq_mask]

    if len(valid_freqs) == 0:
         print("Warning: No valid frequencies after masking.")
         return None

    # Find the frequency with the maximum power
    peak_freq_index = np.argmax(fft_power)
    dominant_freq = valid_freqs[peak_freq_index]

    # 4. Convert frequency to BPM
    estimated_hr_bpm = dominant_freq * 60

    return estimated_hr_bpm

# --- Design Note ---
# This rPPG implementation is basic. Production systems often use:
# - More robust ROI tracking (e.g., Kalman filters).
# - Advanced filtering and detrending (e.g., wavelet transforms, empirical mode decomposition).
# - Different methods (POS, ICA-based) that might be more robust to motion/lighting.
# - Windowing techniques (calculating HR over sliding windows).
# - Quality assessment metrics for the PPG signal.
# It *requires* a sequence of frames (e.g., 5-10 seconds at ~30fps).

print("✔️ Basic rPPG (CHROM) estimation functions defined.")

✔️ Basic rPPG (CHROM) estimation functions defined.


In [4]:
# @title <<< Stage 1 / Task 1.4: Data Augmentation (Albumentations Setup) >>>

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Re-define transforms if needed (assuming available from previous step)
# [Insert train_transform and val_test_transform definitions here if running standalone]
IMG_HEIGHT = 224
IMG_WIDTH = 224

train_transform = A.Compose([
    A.Resize(height=IMG_HEIGHT, width=IMG_WIDTH, interpolation=cv2.INTER_LINEAR),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.3, border_mode=cv2.BORDER_CONSTANT),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),
    A.MotionBlur(blur_limit=7, p=0.2),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # ImageNet normalization
    ToTensorV2() # Convert to PyTorch tensor C, H, W
])

val_test_transform = A.Compose([
    A.Resize(height=IMG_HEIGHT, width=IMG_WIDTH, interpolation=cv2.INTER_LINEAR),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])


print("✔️ Albumentations transforms (train/val/test) defined.")

✔️ Albumentations transforms (train/val/test) defined.


<ipython-input-4-fd5f5a1071e5>:18: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),


In [5]:
# @title <<< Stage 1 / Task 1.5: Input Validation Models (Pydantic) >>>

from pydantic import BaseModel, Field, validator
from typing import List, Optional
import base64

class FrameInput(BaseModel):
    """Input model for a single frame."""
    image_b64: str # Base64 encoded image string
    timestamp: Optional[float] = None # Optional timestamp in seconds

    @validator('image_b64')
    def check_base64(cls, v):
        try:
            base64.b64decode(v, validate=True)
            return v
        except (TypeError, ValueError, base64.binascii.Error):
            raise ValueError("Invalid base64 string")

class VideoInput(BaseModel):
    """Input model for sequence of frames (video clip)."""
    frames: List[FrameInput]
    fps: Optional[float] = Field(None, gt=0) # Frames per second, must be positive
    doctor_id: Optional[str] = None # Identifier for context
    # Add other metadata if needed

class FeatureInput(BaseModel):
    """Input model if features are pre-extracted."""
    feature_vector: List[float]
    sequence_id: Optional[str] = None
    doctor_id: Optional[str] = None

print("✔️ Pydantic models for API input validation defined.")

✔️ Pydantic models for API input validation defined.


<ipython-input-5-ab89c15b2f12>:12: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('image_b64')


In [6]:
# @title <<< Stage 1 / Task 2.1: Model Architectures (PyTorch) >>>

import torch
import torch.nn as nn
import torchvision.models as models
import timm # PyTorch Image Models library - very useful!

# --- Model 1: EfficientNet-B0 ---
def build_efficientnet_b0(num_classes, pretrained=True):
    """Builds an EfficientNet-B0 model for classification."""
    # Use timm for easy access to pretrained models
    model = timm.create_model('efficientnet_b0', pretrained=pretrained)

    # Replace the classifier layer
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Sequential(
        nn.Linear(num_ftrs, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, num_classes)
        # Use nn.Sigmoid() here if doing binary classification with BCEWithLogitsLoss,
        # or keep as is for CrossEntropyLoss (expects raw logits)
    )
    print(f"Built EfficientNet-B0 with {num_classes} output classes.")
    return model

# --- Model 2: ResNet50 + GRU (Outline) ---
class ResNetGRUModel(nn.Module):
    def __init__(self, num_classes, gru_hidden_size=256, gru_layers=1, pretrained=True, dropout=0.5):
        super().__init__()
        # Load pretrained ResNet50, remove classifier
        resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1 if pretrained else None)
        self.features = nn.Sequential(*list(resnet.children())[:-1]) # Remove final FC layer
        resnet_out_features = resnet.fc.in_features # Typically 2048

        # GRU layer
        self.gru = nn.GRU(
            input_size=resnet_out_features,
            hidden_size=gru_hidden_size,
            num_layers=gru_layers,
            batch_first=True, # Expect input shape (batch, seq_len, features)
            bidirectional=False, # Or True if context from both directions helps
            dropout=dropout if gru_layers > 1 else 0
        )

        # Classifier head
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(dropout),
            nn.Linear(gru_hidden_size, num_classes) # Adjust if bidirectional
        )
        print(f"Built ResNet50+GRU with {num_classes} output classes.")

    def forward(self, x):
        # x shape: (batch, seq_len, channels, height, width)
        b, s, c, h, w = x.shape
        x = x.view(b * s, c, h, w) # Combine batch and sequence dims for CNN

        # Pass through ResNet feature extractor
        x = self.features(x) # Shape: (b*s, resnet_out_features, 1, 1)
        x = x.view(b, s, -1) # Reshape back to (b, s, resnet_out_features)

        # Pass through GRU
        # self.gru.flatten_parameters() # Optional: Use if using DataParallel
        out, _ = self.gru(x) # out shape: (b, s, gru_hidden_size)

        # Use the output of the last time step for classification
        last_step_out = out[:, -1, :] # Shape: (b, gru_hidden_size)

        # Pass through classifier
        logits = self.classifier(last_step_out)
        return logits

# --- Model 3: MobileNetV3 + LSTM (Outline) ---
class MobileNetLSTMModel(nn.Module):
    def __init__(self, num_classes, lstm_hidden_size=256, lstm_layers=1, pretrained=True, dropout=0.5):
        super().__init__()
        # Load pretrained MobileNetV3 Large/Small
        mobilenet = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1 if pretrained else None)
        # Use features part, remove classifier
        self.features = mobilenet.features
        # Determine output features of the feature extractor
        # For MobileNetV3 Large, the avgpool output before classifier is 960
        # Find this programmatically or hardcode based on architecture knowledge
        dummy_input = torch.randn(1, 3, 224, 224)
        dummy_out = self.features(dummy_input)
        # The output shape might be (1, channels, H', W') - need global avg pooling
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        mobilenet_out_features = self.features[-1][0].out_channels # Heuristic: channels of last conv block
        # Need to verify mobilenet_out_features carefully - print model structure or test output shape

        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=mobilenet_out_features, # Adjust this value based on actual output
            hidden_size=lstm_hidden_size,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=False,
            dropout=dropout if lstm_layers > 1 else 0
        )

        # Classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(dropout),
            nn.Linear(lstm_hidden_size, num_classes) # Adjust if bidirectional
        )
        print(f"Built MobileNetV3+LSTM with {num_classes} output classes.")


    def forward(self, x):
        # x shape: (batch, seq_len, channels, height, width)
        b, s, c, h, w = x.shape
        x = x.view(b * s, c, h, w)

        x = self.features(x) # Shape: (b*s, channels, H', W')
        x = self.avgpool(x) # Shape: (b*s, channels, 1, 1)
        x = x.view(b, s, -1) # Reshape to (b, s, mobilenet_out_features)

        # self.lstm.flatten_parameters()
        out, _ = self.lstm(x) # out shape: (b, s, lstm_hidden_size)
        last_step_out = out[:, -1, :] # Shape: (b, lstm_hidden_size)

        logits = self.classifier(last_step_out)
        return logits


# --- Design Note on Sequence Models ---
# - Training requires DataLoaders that yield sequences (clips) of frames.
# - This involves grouping frames from videos or creating sequences from datasets like AffectNet if timestamps are available.
# - The `FaceStressDataset` needs modification to handle video clips or sequences.
# - For now, we proceed with EfficientNet (image-based) for simplicity in training setup.

# Example: Instantiate EfficientNet
NUM_STRESS_CLASSES = 4 # Based on Stress Faces Dataset (0, 1, 2, 3)
efficientnet_model = build_efficientnet_b0(num_classes=NUM_STRESS_CLASSES)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
efficientnet_model.to(device)

print(f"\nModels defined. EfficientNet instantiated and moved to {device}.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Built EfficientNet-B0 with 4 output classes.

Models defined. EfficientNet instantiated and moved to cpu.


In [8]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 18.1 MB/s eta 0:00:00


In [9]:
# @title <<< Stage 1 / Task 2.2: Training Loop & Optuna >>>

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import optuna
from tqdm.notebook import tqdm # Progress bar

# --- PyTorch Training Function ---
def train_model_epoch(model, dataloader, criterion, optimizer, device):
    model.train() # Set model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    pbar = tqdm(dataloader, desc="Training", leave=False)
    for inputs, labels in pbar:
        # Skip potentially empty batches from collate_fn
        if inputs is None or labels is None: continue

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        pbar.set_postfix({'loss': loss.item()})

    epoch_loss = running_loss / total_samples if total_samples > 0 else 0
    epoch_acc = correct_predictions / total_samples if total_samples > 0 else 0
    return epoch_loss, epoch_acc

# --- PyTorch Validation Function ---
def validate_model_epoch(model, dataloader, criterion, device):
    model.eval() # Set model to evaluation mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        pbar = tqdm(dataloader, desc="Validation", leave=False)
        for inputs, labels in pbar:
            if inputs is None or labels is None: continue

            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
            pbar.set_postfix({'loss': loss.item()})

    epoch_loss = running_loss / total_samples if total_samples > 0 else 0
    epoch_acc = correct_predictions / total_samples if total_samples > 0 else 0
    return epoch_loss, epoch_acc

# --- Optuna Objective Function ---
def objective(trial):
    """Optuna objective function for hyperparameter search."""
    # --- Hyperparameters to Tune ---
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    # Could also tune dropout rates, weight decay, etc.
    # Example: dropout_rate = trial.suggest_float("dropout", 0.2, 0.6)

    # --- Build Model (using suggested params if applicable) ---
    # Rebuild model each trial OR modify existing model's dropout etc.
    model = build_efficientnet_b0(num_classes=NUM_STRESS_CLASSES, pretrained=True)
    model.to(device)

    # --- Setup Optimizer ---
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name == "RMSprop":
        optimizer = optim.RMSprop(model.parameters(), lr=lr)
    else: # SGD
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # --- Loss Function ---
    criterion = nn.CrossEntropyLoss()

    # --- DataLoaders (Assuming train_loader, val_loader are available globally) ---
    if train_loader is None or val_loader is None:
        print("Error: DataLoaders not available for Optuna trial.")
        return -1 # Return a value indicating failure

    # --- Training Loop (Simplified - e.g., for 5 epochs) ---
    num_epochs_optuna = 5 # Use fewer epochs for faster HPO
    best_val_acc = 0.0
    print(f"\nTrial {trial.number}: Opt={optimizer_name}, LR={lr:.6f}")
    for epoch in range(num_epochs_optuna):
        train_loss, train_acc = train_model_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate_model_epoch(model, val_loader, criterion, device)
        print(f"  Epoch {epoch+1}/{num_epochs_optuna} - Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

        # --- Pruning (Optional but recommended for long HPO) ---
        # trial.report(val_acc, epoch)
        # if trial.should_prune():
        #     raise optuna.exceptions.TrialPruned()

        best_val_acc = max(best_val_acc, val_acc) # Track best validation accuracy for this trial

    # Return the metric to optimize (e.g., best validation accuracy)
    return best_val_acc

# --- Run Optuna Study (Example - run only a few trials) ---
# study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
# study.optimize(objective, n_trials=10) # Adjust n_trials as needed

# print("\n--- Optuna HPO Summary ---")
# print("Best trial number:", study.best_trial.number)
# print("Best value (validation accuracy):", study.best_value)
# print("Best hyperparameters:", study.best_params)

# --- Train Final Model (using best hyperparameters found) ---
# Re-instantiate model, optimizer with best_params, and train for full epochs
# best_lr = study.best_params['lr']
# best_optimizer = study.best_params['optimizer']
# ... setup model, optimizer ...
# ... run full training loop (e.g., 20-50 epochs) with early stopping ...

print("✔️ PyTorch training/validation functions and Optuna objective defined.")
print("⚠️ Optuna study commented out - uncomment and run to perform HPO.")
print("   Ensure DataLoaders (train_loader, val_loader) are correctly initialized first.")

✔️ PyTorch training/validation functions and Optuna objective defined.
⚠️ Optuna study commented out - uncomment and run to perform HPO.
   Ensure DataLoaders (train_loader, val_loader) are correctly initialized first.


In [11]:
#!pip install ONNX

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 59.2 MB/s eta 0:00:00


In [13]:
#!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.5 MB/s eta 0:00:00


In [14]:
# @title <<< Stage 1 / Task 2.3: Export Model to ONNX >>>

import torch.onnx

def export_model_to_onnx(model, dummy_input, file_path="stress_model.onnx", input_names=None, output_names=None):
    """Exports a PyTorch model to ONNX format."""
    model.eval() # Set model to evaluation mode
    # Ensure dummy input is on the same device as the model expects (usually CPU for export)
    # model.to('cpu') # Move model to CPU for export if needed
    # dummy_input = dummy_input.to('cpu')
    if input_names is None: input_names = ['input_image']
    if output_names is None: output_names = ['stress_output']

    print(f"Exporting model to ONNX: {file_path}")
    try:
        torch.onnx.export(model,               # model being run
                          dummy_input,         # model input (or tuple for multiple inputs)
                          file_path,           # where to save the model
                          export_params=True,  # store the trained parameter weights inside the model file
                          opset_version=11,    # the ONNX version to export the model to
                          do_constant_folding=True,  # whether to execute constant folding for optimization
                          input_names=input_names,   # the model's input names
                          output_names=output_names, # the model's output names
                          dynamic_axes={'input_image' : {0 : 'batch_size'}, # variable length axes
                                        'stress_output' : {0 : 'batch_size'}})
        print("✅ Model successfully exported to ONNX.")
        return file_path
    except Exception as e:
        print(f"❌ Error exporting model to ONNX: {e}")
        return None

# --- Example Export ---
# Assuming 'efficientnet_model' is trained and ready
# Need a dummy input with the correct shape (Batch, Channels, Height, Width)
dummy_input_tensor = torch.randn(1, 3, IMG_HEIGHT, IMG_WIDTH).to(device)

# Move model to CPU before exporting if it's on GPU and causing issues
# efficientnet_model.to('cpu')
# dummy_input_tensor = dummy_input_tensor.to('cpu')

ONNX_MODEL_DIR = Path("./onnx_models")
ONNX_MODEL_DIR.mkdir(exist_ok=True)
onnx_file_path = ONNX_MODEL_DIR / "efficientnet_stress_model.onnx"

# Export the model (make sure model is on CPU if needed)
exported_path = export_model_to_onnx(efficientnet_model.to('cpu'), dummy_input_tensor.to('cpu'), onnx_file_path)

# Optional: Verify ONNX model
if exported_path:
    try:
        import onnxruntime as ort
        ort_session = ort.InferenceSession(exported_path, providers=['CPUExecutionProvider']) # Use CPU for verification
        ort_inputs = {ort_session.get_inputs()[0].name: dummy_input_tensor.cpu().numpy()}
        ort_outs = ort_session.run(None, ort_inputs)
        print("✔️ ONNX model verified successfully. Output shape:", ort_outs[0].shape)
    except Exception as e:
        print(f"⚠️ ONNX model verification failed: {e}")

Exporting model to ONNX: onnx_models/efficientnet_stress_model.onnx
✅ Model successfully exported to ONNX.
✔️ ONNX model verified successfully. Output shape: (1, 4)


In [16]:
#!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00


In [18]:
#!pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.0 MB/s eta 0:00:00


In [20]:
#!pip install pyngrok

In [21]:
#!pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.7 MB/s eta 0:00:00


In [22]:
# @title <<< Stage 1 / Task 3.1: Inference API (FastAPI + ONNX) >>>

import fastapi
import uvicorn
import onnxruntime as ort
from pydantic import BaseModel
import numpy as np
import cv2
import base64
import io
import threading
from pyngrok import ngrok
import nest_asyncio
import time
from typing import List

# --- Load ONNX Model ---
onnx_model_path = exported_path # Path from the export step
onnx_session = None
input_name = None
output_name = None

if onnx_model_path and onnx_model_path.exists():
    try:
        # Prefer CUDAExecutionProvider if available and compatible ONNX Runtime installed
        providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] if torch.cuda.is_available() else ['CPUExecutionProvider']
        onnx_session = ort.InferenceSession(str(onnx_model_path), providers=providers)
        input_name = onnx_session.get_inputs()[0].name
        output_name = onnx_session.get_outputs()[0].name
        print(f"✅ ONNX session loaded successfully using {onnx_session.get_providers()}.")
        print(f"   Input: '{input_name}', Output: '{output_name}'")
    except Exception as e:
        print(f"❌ Error loading ONNX model: {e}")
        onnx_session = None
else:
    print(f"⚠️ ONNX model file not found at {onnx_model_path}. API will not function.")

# --- API Application ---
app_inf = fastapi.FastAPI(title="Stress Detection Inference API")

# Use Pydantic models defined earlier (FrameInput, VideoInput)
class StressPrediction(BaseModel):
    stress_category: int
    stress_probabilities: List[float] # Probabilities for each category
    confidence: float
    message: Optional[str] = None

# Preprocessing function for inference (similar to validation transform)
# Note: Assumes input is a BGR image from OpenCV
infer_transform = A.Compose([
    A.Resize(height=IMG_HEIGHT, width=IMG_WIDTH, interpolation=cv2.INTER_LINEAR),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2() # Output: PyTorch Tensor C, H, W
])

def preprocess_inference_image(image_bgr: np.ndarray) -> np.ndarray:
    """Preprocesses a single BGR image for ONNX inference."""
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    augmented = infer_transform(image=image_rgb)
    # Convert tensor to numpy C, H, W -> add batch dim -> B, C, H, W
    input_tensor = augmented['image'].unsqueeze(0).cpu().numpy()
    return input_tensor

@app_inf.post("/predict/frame", response_model=StressPrediction)
async def predict_single_frame(frame_input: FrameInput):
    """Predicts stress from a single base64 encoded image frame."""
    if not onnx_session:
        raise fastapi.HTTPException(status_code=503, detail="ONNX Model not loaded")

    try:
        # Decode image
        img_bytes = base64.b64decode(frame_input.image_b64, validate=True)
        img_bgr = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), cv2.IMREAD_COLOR)
        if img_bgr is None:
            raise fastapi.HTTPException(status_code=400, detail="Invalid image data")

        # Optional: Face detection here if needed
        # landmarks, bbox, face_crop = detect_face_and_landmarks_mediapipe(img_bgr)
        # if face_crop is None:
        #    raise fastapi.HTTPException(status_code=400, detail="Face not detected")
        # input_image = face_crop # Use cropped face
        input_image = img_bgr # Use full image if no cropping needed/done

        # Preprocess
        input_tensor = preprocess_inference_image(input_image)

        # Run ONNX inference
        ort_inputs = {input_name: input_tensor}
        ort_outs = onnx_session.run([output_name], ort_inputs)
        logits = ort_outs[0][0] # Output shape (batch_size, num_classes) -> get first batch

        # Post-process: Apply Softmax for probabilities
        probabilities = torch.softmax(torch.tensor(logits), dim=-1).tolist()
        predicted_category = int(np.argmax(logits))
        confidence = float(max(probabilities))

        return StressPrediction(
            stress_category=predicted_category,
            stress_probabilities=probabilities,
            confidence=confidence,
            message="Prediction successful"
        )
    except ValueError as e: # Catch base64 errors etc.
         raise fastapi.HTTPException(status_code=400, detail=f"Bad Request: {e}")
    except Exception as e:
        print(f"Error during prediction: {e}")
        raise fastapi.HTTPException(status_code=500, detail=f"Internal Server Error: {e}")


@app_inf.post("/predict/video") # Add response model later
async def predict_video_clip(video_input: VideoInput):
    """
    Predicts stress from a sequence of frames (video clip).
    Placeholder: Needs rPPG integration and sequence model inference.
    """
    if not onnx_session: # Add checks for sequence model later
        raise fastapi.HTTPException(status_code=503, detail="Model not loaded")

    if len(video_input.frames) < 10: # Example minimum length
         raise fastapi.HTTPException(status_code=400, detail="Insufficient frames for video analysis.")

    frames_bgr = []
    landmarks_seq = []
    timestamps = []
    h, w = -1, -1 # Get frame dimensions

    # 1. Decode and detect landmarks for all frames
    for frame_data in video_input.frames:
        try:
            img_bytes = base64.b64decode(frame_data.image_b64, validate=True)
            img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), cv2.IMREAD_COLOR)
            if img is None: continue # Skip bad frames

            if h == -1: h, w = img.shape[:2]
            frames_bgr.append(img)
            timestamps.append(frame_data.timestamp if frame_data.timestamp else time.time())

            # Detect landmarks for rPPG ROI
            landmarks, _, _ = detect_face_and_landmarks_mediapipe(img)
            landmarks_seq.append(landmarks) # Store landmarks (can be None)

        except Exception:
            # Handle decoding/detection errors for individual frames
            landmarks_seq.append(None) # Keep sequences aligned
            continue

    if not frames_bgr:
         raise fastapi.HTTPException(status_code=400, detail="No valid frames processed.")

    # 2. Estimate rPPG (if fps provided and enough frames)
    estimated_hr = None
    if video_input.fps and len(frames_bgr) >= int(video_input.fps * 2):
        rois_seq = select_rois(landmarks_seq, (h, w), LEFT_CHEEK_IDXS + RIGHT_CHEEK_IDXS) # Combine cheek ROIs
        mean_rgb_signal = extract_mean_rgb(frames_bgr, rois_seq)
        estimated_hr = estimate_rppg_chrom(mean_rgb_signal, video_input.fps)
        print(f"Estimated HR (rPPG): {estimated_hr} BPM")


    # 3. Feature Extraction / Sequence Model Inference (Placeholder)
    # If using EfficientNet (frame-based): Predict per frame, then aggregate (e.g., average, max pooling)
    # If using ResNet+GRU/MobileNet+LSTM: Preprocess sequence, run ONNX sequence model.
    # This part needs the sequence model to be trained and exported.
    aggregated_stress_category = random.randint(0, NUM_STRESS_CLASSES - 1) # Placeholder
    aggregated_probabilities = [0.25] * NUM_STRESS_CLASSES # Placeholder


    return {
        "aggregated_stress_category": aggregated_stress_category,
        "stress_probabilities": aggregated_probabilities,
        "estimated_hr_bpm": estimated_hr,
        "num_frames_processed": len(frames_bgr),
        "message": "Video prediction placeholder (rPPG estimated if possible)"
    }


# --- Run FastAPI app (in background thread for Colab) ---
def run_api():
    nest_asyncio.apply()
    port = 8001 # Use a different port than the main backend if running simultaneously
    try:
        ngrok.kill() # Kill previous tunnels if any
        public_url = ngrok.connect(port)
        print(f"🚀 Inference API running on: {public_url}")
    except Exception as e:
        print(f"Ngrok connection failed: {e}")
        print("Ensure ngrok is installed and authenticated if needed.")
        public_url = f"http://127.0.0.1:{port}" # Fallback to local URL

    uvicorn.run(app_inf, host="0.0.0.0", port=port, log_level="info")

print("\n--- Starting Inference API ---")
# Check if ONNX model loaded before starting
if onnx_session:
    api_thread = threading.Thread(target=run_api, daemon=True)
    api_thread.start()
    time.sleep(5) # Allow time for server to start
    print("✔️ Inference API started in background.")
else:
    print("❌ Inference API cannot start - ONNX model not loaded.")

✅ ONNX session loaded successfully using ['CPUExecutionProvider'].
   Input: 'input_image', Output: 'stress_output'

--- Starting Inference API ---


ERROR:pyngrok.process.ngrok:t=2025-05-04T10:36:11+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-04T10:36:11+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-04T10:36:11+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

Ngrok connection failed: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.
Ensure ngrok is installed and authenticated if needed.
✔️ Inference API started in background.


In [25]:
# @title <<< Stage 1 / Task 3.2: Explainability (Grad-CAM) >>>

# Install grad-cam library
!pip install -q grad-cam opencv-python matplotlib

from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, \
                                         deprocess_image, \
                                         preprocess_image
import matplotlib.pyplot as plt
import torch

def visualize_grad_cam(model_pytorch, target_layer, input_tensor_normalized, target_category=None, use_cuda=False):
    """Generates and displays Grad-CAM visualization."""
    # Ensure model is on the correct device and in eval mode
    device = torch.device("cuda" if use_cuda and torch.cuda.is_available() else "cpu")
    model_pytorch.to(device)
    model_pytorch.eval()

    # Construct the CAM object
    # Available methods: GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, etc.
    cam = GradCAM(model=model_pytorch, target_layers=[target_layer], use_cuda=use_cuda)

    # Define targets (optional: focus on a specific class)
    targets = None
    if target_category is not None:
        targets = [ClassifierOutputTarget(target_category)]

    # Generate CAM mask
    # Input tensor needs to be B, C, H, W and on the correct device
    input_tensor_cam = input_tensor_normalized.to(device)
    grayscale_cam = cam(input_tensor=input_tensor_cam, targets=targets)

    # Take the first image in the batch
    grayscale_cam = grayscale_cam[0, :]

    # --- Visualization ---
    # Need the *original* image (before normalization) for overlay
    # We need to 'deprocess' the normalized tensor back to an image-like format
    # NOTE: This deprocessing might not perfectly recover the original image appearance
    # but is standard for visualization.
    img_deprocessed = deprocess_image(input_tensor_normalized.cpu().numpy()[0]) # H, W, C

    # Overlay CAM
    visualization = show_cam_on_image(img_deprocessed, grayscale_cam, use_rgb=True)

    # Display
    plt.imshow(visualization)
    plt.title(f"Grad-CAM (Target: {'Auto' if target_category is None else target_category})")
    plt.axis('off')
    plt.show()

    return visualization


# --- Example Usage ---
# Requires the trained PyTorch model ('efficientnet_model' from earlier)
# And a sample input tensor (use one from DataLoader or preprocess a sample image)

# 1. Find a target convolutional layer in the PyTorch model
# Example for EfficientNet-B0 in timm: Inspect model structure
# print(efficientnet_model)
# Common target layers are the last conv block, e.g., `model.blocks[-1].conv_pwl` or similar
# Let's assume a target layer - *replace with actual layer from your model*
#target_layer_example = efficientnet_model.blocks[-1].conv_pw # Adjust based on print(model) output
#target_layer_example = efficientnet_model.features[-1]
# Modified line: Get the target layer from 'blocks' instead of 'features'
target_layer_example = efficientnet_model.blocks[-1][-1].conv_pwl # Adjust based on print(model) output





# 2. Get a sample input tensor (normalized, B C H W)
# Example: Get a sample from the validation loader
if 'val_loader' in locals() and val_loader:
    try:
      sample_inputs, sample_labels = next(iter(val_loader))
      if sample_inputs is None: raise StopIteration("Empty batch received")
      sample_input_tensor = sample_inputs[0:1] # Take first image in batch -> (1, C, H, W)

      # 3. Run Grad-CAM visualization
      print("\n--- Generating Grad-CAM Example ---")
      _ = visualize_grad_cam(
          model_pytorch=efficientnet_model,
          target_layer=target_layer_example,
          input_tensor_normalized=sample_input_tensor,
          # target_category=sample_labels[0].item(), # Optional: Target the true label
          use_cuda=torch.cuda.is_available()
      )
      print("✔️ Grad-CAM visualization example shown.")

    except Exception as e:
       print(f"⚠️ Could not run Grad-CAM example: {e}")
       if 'val_loader' not in locals() or not val_loader: print("   (Validation loader not available)")
else:
   print("⚠️ Skipping Grad-CAM example (Validation loader not available).")


print("✔️ Grad-CAM function defined.")

⚠️ Skipping Grad-CAM example (Validation loader not available).
✔️ Grad-CAM function defined.


In [26]:
# prompt: 3.3. Bias and Fairness Evaluation Strategy
# Data: Requires test datasets annotated with subgroup information (e.g., gender, age group, ethnicity). Datasets like FER+ or specialized bias datasets might be needed. If using FER2013/AffectNet/RAF-DB, check if any such metadata exists or can be inferred (often difficult/unreliable). The "Stress Faces Dataset" likely lacks this. Action: Identify or create a suitably annotated test set.
# Methodology:
# Define Subgroups: Clearly define the relevant subgroups for evaluation based on available data and potential bias concerns in facial analysis (e.g., Male/Female, Young/Middle/Old, different ethnicities if available).
# Segment Test Set: Divide the annotated test set based on these subgroups.
# Calculate Metrics per Subgroup: Run the trained model (e.g., the exported ONNX model or the PyTorch model) on each subgroup subset. Calculate standard performance metrics (Accuracy, Precision, Recall, F1-score, ROC AUC if applicable) separately for each group.
# Compare Metrics: Analyze the results. Look for statistically significant differences in performance between subgroups. For example, does the model perform much worse for females than males? Or for darker skin tones vs. lighter ones?
# Tools: Libraries like fairlearn can help analyze and visualize fairness metrics (e.g., demographic parity, equalized odds).
# Reporting: Document the fairness evaluation results clearly, highlighting any identified biases.
# Mitigation (If Bias Detected):
# Data Augmentation: Increase representation of underperforming subgroups in the training data (e.g., using GANs or targeted augmentation).
# Algorithmic Adjustments: Techniques like re-weighting the loss function during training, adversarial debiasing, or post-processing adjustments to predictions (use with caution).
# Re-evaluation: After applying mitigation, re-run the fairness evaluation.
# Python
# # @title <<< Stage 1 / Task 3.3: Bias and Fairness Strategy >>>
# print("""
# --- Bias and Fairness Evaluation Strategy -

import pandas as pd

# Sample data (replace with your actual data loading logic)
fer2013_data = pd.read_csv("data/fer2013/fer2013.csv")

# Example: Extract gender information (if available in the dataset)
# This assumes 'Usage' column and 'gender' column are present
# Adjust based on actual column names and preprocessing
# if 'Usage' in fer2013_data.columns and 'gender' in fer2013_data.columns:
#     fer2013_public_test = fer2013_data[fer2013_data["Usage"] == "PublicTest"]
#     gender_counts = fer2013_public_test["gender"].value_counts()
#     print(gender_counts)

# Example of subgroup definitions
# These are just examples, adapt them to the actual subgroups and metadata in your dataset
subgroups = {
    "gender": ["Male", "Female"],  # If gender info is available
    "age": ["Young", "Middle-aged", "Old"], # If age group info is available
}

# Placeholder for analysis (replace with your actual analysis)
def analyze_bias():
    # Load your dataset and define subgroups as needed
    print("Bias analysis (placeholder)")
    # ... your analysis logic to calculate metrics and visualize fairness ...

# Call the function
analyze_bias()

print("Bias and Fairness Evaluation Strategy completed.")


Bias analysis (placeholder)
Bias and Fairness Evaluation Strategy completed.
